<center> <h1>libOmexMeta Supplementary Content </h1> </center>
This document describes how to use libOmexMeta version 1.1. We split this document
into two sections. In part A we provide a tutorial in which we focus on providing use case examples
for each type of annotation supported with a narrative to explain the annotations. Part B is example
focused and demonstrates for programmers how one is likely to use pyomexmeta, the Python front end
of libOmexMeta. Please note that throughout this document we use examples that are intended to be illustrative
with regards to annotation, but not biologically accurate or entirely realistical modelling scenarios.

We further wish to emphasize that the users of OMEX metadata libraries are programmers,
rather than biologists. As described in the manuscript, if tool developers use libOmexMeta or
pyomexmeta libraries, then their tools, which are used by biologists, will produce annotations on models
that are compliant with the standard, and therefore will better support findability,
accessibility, interoperability, and reusability of those models.

<center> <h2>Part A: libOmexMeta Tutorial</h2></center>

<left> <h3> 1. Model-level annotations </h3> </left>
We begin by demonstrating how to create annotations to indicate the creator,
curator, taxon, publication ID, and date created for an sbml model. The SBML model that we use in this example is nothing
more than a basic toy model where A transitions to B using first order mass action kinetics.

In [2]:
# example 1

from pyomexmeta import RDF, eUriType

sbml1 = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="ToyModel" id="ToyModel">
    <listOfCompartments>
      <compartment sboTerm="SBO:0000410" id="cell" metaid="cell" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="A" metaid="A" compartment="cell" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="B" metaid="B" compartment="cell" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="k1" value="0.1" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="A" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="B" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> k1 </ci>
              <ci> A </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


rdf_graph1 = RDF()
rdf_graph1.set_archive_uri("MyOmexArchive.omex")
rdf_graph1.set_model_uri("MyModel.sbml")
annot_editor = rdf_graph1.to_editor(sbml1, generate_new_metaids=False, sbml_semantic_extraction=False)
annot_editor.add_creator("orcid:0000-0001-8254-4957") \
    .add_curator("orcid:0000-0001-8254-4958") \
    .add_taxon("taxon/9895") \
    .add_pubmed("pubmed/12334") \
    .add_date_created("18-09-2020")

print(rdf_graph1)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix pubmed: <https://identifiers.org/pubmed:> .
@prefix bqmodel: <http://biomodels.net/model-qualifiers/> .
@prefix NCBI_Taxon: <https://identifiers.org/taxonomy:> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix dc: <https://dublincore.org/specifications/dublin-core/dcmi-terms/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/MyOmexArchive.omex/> .
@prefix local: <http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#>
    dc:creator <https://orchid.org/orcid:0000-0001-8254-4958> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.sbml#>
    bqbiol:hasTaxon <https://identifiers.org/taxonomy:taxon/9895> ;
    bqmodel:isDescribedBy <https://identifiers.org/pubmed:pubmed/12334> ;
    dc:created [
        dc:W3CDTF "18-09-2020"^^rdf:string
    ] ;
    dc:creator <https://orchid.org/orcid:0000-0001-8

As described in the OmexMeta v1.1 specification, it is also optionally possible to specify
the people (curators, creators) with strings for names, email addresses, etc.

<left> <h3> 2. Singular Annotations </h3> </left>
In this section, we demonstrate how one can create a single annotation, which is an RDF triple
object comprising a subject, a predicate and an object. We reuse the already defined `sbml` string
variable from example 1.1.

In [4]:
# Example 1
from pyomexmeta import RDF, eUriType

sbml2 = sbml1
rdf_graph2 = RDF()
rdf_graph2.set_archive_uri("MyOmexArchive.omex")
rdf_graph2.set_model_uri("MyModel.sbml")
annot_editor = rdf_graph2.to_editor(sbml2, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_singular_annotation() as singular_annotation:
    # CHEBI:16236 = EtOH in chebi
     singular_annotation.about("A") \
                        .predicate("bqbiol", "is") \
                        .resource_uri("CHEBI:16236")
print(rdf_graph2)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/MyOmexArchive.omex/> .
@prefix local: <http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.sbml#A>
    bqbiol:is <https://identifiers.org/CHEBI:16236> .




This example shows the lowest level interface for creating an annotation. These three methods (`about()`, `predicate()` and `resource_uri()`)
demonstrate how to create the [subject, predicate and resource](https://sys-bio.github.io/libOmexMeta/docs-build/singular_annotations/index.html#subjects-predicates-and-resources)
parts of an RDF triple object respectively. Note, that there also exists a `resource_literal()` and a `resource_blank()`
for creating RDF resources that are literal or blank nodes respectively. This annotation states that the SBML species
with metaid `A` is th chemical entity that has the CHEBI identifier `CHEBI:16236`, which is ethanol.

<left> <h3> 3. Composite Annotations </h3> </left>

Composite annotations are collections of singlular annotations with a predefined structure. Three types
of composite annotation are supported by libOmexMeta v1.1: 1) annotation for a physical entity; 2) a physical process
and 3) an energy differential. In this section we describe these further using examples in both SBML and CellML.
<left> <h4> 3.1 SBML </h4> </left>


<left> <h5> 3.1.1 Annotations for a Physical Entity </h5> </left>

In many modeling situations, one may wish to annotate both the physical entity and the physical property that
is being simulated, such as concentration, particle numbers, fluid volume or fluid pressure.
This allows us to capture more complete biological semantics. For instance, in a cardiovascular fluid flow model, one can model
either the fluid volume or fluid pressure inside the left ventricle. In chemical
kinetic models, one can model the chemical concentration or the amount in absolute numbers of a physical
entity, such as glucose or ethanol.

In some modeling languages, only one of these aspects may be present in the code.
For example, in CellML, one has variables that represent chemical concentrations,
or blood pressures, but the biological physical entity is implicit. Conversely, in SBML,
the physical entity is specified as a “species”, but the physical property (usually chemical
concentration or particle numbers) is left implicit. Thus, for annotation, we must allow for either situation.

A physical entity may be a complex structured object. For example, one may need to distinguish
between nuclear and cytoplasmic glucose. These are both the same entity (glucose), but
they are part of different anatomic structures. In the following example we demonstrate how
to annotate such a situation using another toy model depicting `glucose` in dynamic equilibrium
between two compartments, the `nucleus` and the `cytoplasm`.

In [5]:
# Example 3.1.1
from pyomexmeta import RDF, eUriType

sbml3_1_1 = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="GlucoseTransport" id="GlucoseTransport">
    <listOfCompartments>
      <compartment id="cytoplasm" metaid="cytoplasm" spatialDimensions="3" size="1" constant="true"/>
      <compartment id="nucleus" metaid="nucleus" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="glucose_c" metaid="glucose_c" compartment="cytoplasm" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="glucose_n" metaid="glucose_n" compartment="nucleus" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kimp" metaid="kimp" value="10" constant="true"/>
      <parameter id="kexp" metaid="kexp" value="0.1" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="r1_imp"  metaid="r1_imp" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="glucose_c" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="glucose_n" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> cytoplasm </ci>
              <ci> kimp </ci>
              <ci> glucose_c </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="r2_exp" metaid="r2_exp" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="glucose_n" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="glucose_c" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> nucleus </ci>
              <ci> kexp </ci>
              <ci> glucose_n </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""

rdf_graph3_1_1 = RDF()
rdf_graph3_1_1.set_archive_uri("GlucoseTransport3_1_1.omex")
rdf_graph3_1_1.set_model_uri("glucose_transport3_1_1.sbml")
annot_editor = rdf_graph3_1_1.to_editor(sbml3_1_1, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = GO cellular component term for cytoplasm.
    # OPB:00340 = OPB term for chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005737")   \
                   .has_property("OPB:00340")

with annot_editor.new_physical_entity() as nuclear_glucose:
    # CHEBI:17234 = The GO cellular component term for nucleus.
    nuclear_glucose.about("glucose_n", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005634") \
                   .has_property("OPB:00340")
    
print(rdf_graph3_1_1)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/GlucoseTransport3_1_1.omex/> .
@prefix local: <http://omex-library.org/GlucoseTransport3_1_1.omex/glucose_transport3_1_1.rdf#> .

local:EntityProperty0000
    bqbiol:isPropertyOf <http://omex-library.org/GlucoseTransport3_1_1.omex/glucose_transport3_1_1.sbml#glucose_c> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .

local:EntityProperty0001
    bqbiol:isPropertyOf <http://omex-library.org/GlucoseTransport3_1_1.omex/glucose_transport3_1_1.sbml#glucose_n> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .

<http://omex-library.org/GlucoseTransport3_1_1.omex/glucose_transport3_1_1.sbml#glucose_c>
    bqbiol:is <https://identifiers.org/CHEBI:17234> ;
    bqbiol:isPartOf <https://identifiers.org/GO:0005737> .

<http://omex-library.org/GlucoseTransport3_1_1.o

In this example we construct two physical entity annotations, one each for cytoplasmic and nuclear glucose. The
argument to the `about()` method is the metaid for the SBML species element called `glucose_c`.

      <species id="glucose_c" metaid="glucose_c" compartment="cytoplasm" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>

Internally, libOmexMeta creates a new RDF subject node `local:EntityProperty0000` which is `local` to the annotation document we are creating.

    local:EntityProperty0000
        bqbiol:is <https://identifiers.org/CHEBI:17234> ;
        bqbiol:isPartOf <https://identifiers.org/GO:0005737> .

Often, the `is_part_of` relation is only important when entities can exist in multiple compartments and the modeler needs to distinguish between them.

The `has_property` relation may be used to indicate the property we are simulating. In this case, we are simulating the chemical concentration of the glucose entity. The physical property is optional.

In [14]:
# Example 3.1.1
from pyomexmeta import RDF, eUriType

# reusing sbml from example 3.1.1
sbml3_1_2 = sbml3_1_1

rdf_graph3_1_2 = RDF()
rdf_graph3_1_2.set_archive_uri("GlucoseTransport3_1_2.omex")
rdf_graph3_1_2.set_model_uri("glucose_transport3_1_2.sbml")
# resuing the sbml variable from the previous section
annot_editor = rdf_graph3_1_2.to_editor(sbml3_1_2, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    # OPB:00340 = chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                     .identity("CHEBI:17234")\
                     .is_part_of("GO:0005737")

print(rdf_graph3_1_2)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/GlucoseTransport3_1_2.omex/> .
@prefix local: <http://omex-library.org/GlucoseTransport3_1_2.omex/glucose_transport3_1_2.rdf#> .

<http://omex-library.org/GlucoseTransport3_1_2.omex/glucose_transport3_1_2.sbml#glucose_c>
    bqbiol:is <https://identifiers.org/CHEBI:17234> ;
    bqbiol:isPartOf <https://identifiers.org/GO:0005737> .




Note that the difference between this and the last example is the absence of a `bqbiol:isVersionOf` predicate:

    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .


<left> <h5> 3.1.2 Annotations for a Physical Process </h5> </left>
Perhaps the most prevalent type of biosimulation model is one that simulates biochemical reactions.
Thus, we provide special attention to annotations for biological processes, such as reactions.
Unlike physical entities, processes rarely have unique names. However, some processes can
be categorized as a "version of" a Gene Ontology process term. Thus, in the main text, we used
a "version of" alcohol dehydrogenase. Alcohol dehydrogenase `ADH` is the enzyme that catalyzes the conversion of ethanol
`EtOH` into an aldehyde `Aldehyde` entity. Moreover, NAD+ gains an a H+ in the process as well as two electrons,
thus neutralizing the charge on NAHD, the second product of this reaction. This reaction is depicted
in the sbml model below with simple mass action kinetics.

In [6]:
# Example 3.1.2.1
from pyomexmeta import RDF, eUriType

sbml3_1_2_1 = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
    <model metaid="ADHModel" id="ADHModel">
        <listOfCompartments>
            <compartment id="cytosol" metaid="cytosol" spatialDimensions="3" size="1" constant="true"/>
        </listOfCompartments>
        <listOfSpecies>
            <species id="NAD" metaid="NAD" compartment="cytosol" initialConcentration="10" hasOnlySubstanceUnits="false"
                     boundaryCondition="false" constant="false"/>
            <species id="EtOH" metaid="EtOH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="NADH" metaid="NADH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="Aldehyde" metaid="Aldehyde" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="ADH" metaid="ADH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
        </listOfSpecies>
        <listOfParameters>
            <parameter id="vmax" metaid="k" value="0.5" constant="true"/>
        </listOfParameters>
        <listOfReactions>
            <reaction id="ADHForwardReaction" metaid="ADHForwardReaction" reversible="false" fast="false">
                <listOfReactants>
                    <speciesReference species="EtOH" stoichiometry="1" constant="true"/>
                    <speciesReference species="NAD" stoichiometry="1" constant="true"/>
                </listOfReactants>
                <listOfProducts>
                    <speciesReference species="Aldehyde" stoichiometry="1" constant="true"/>
                    <speciesReference species="NADH" stoichiometry="1" constant="true"/>
                </listOfProducts>
                <kineticLaw>
                    <math xmlns="http://www.w3.org/1998/Math/MathML">
                        <apply>
                            <times/>
                            <ci>k</ci>
                            <ci>ADH</ci>
                            <ci>EtOH</ci>
                            <ci>NAD</ci>
                        </apply>
                    </math>
                </kineticLaw>
            </reaction>
        </listOfReactions>
    </model>
</sbml>
"""

rdf_graph3_1_2_1 = RDF()
rdf_graph3_1_2_1.set_archive_uri("AHD.omex")
rdf_graph3_1_2_1.set_model_uri("AHDModel.sbml")
annot_editor = rdf_graph3_1_2_1.to_editor(sbml3_1_2_1, generate_new_metaids=False, sbml_semantic_extraction=False)

# OPB:OPB_00237 = Chemical flow rate
# GO:0004022    = GO term for Alcohol Dehydrogenase
with annot_editor.new_physical_process() as phy_process:
    phy_process \
        .about("ADHForwardReaction", eUriType.MODEL_URI) \
        .is_version_of("GO:0004022") \
        .add_source(physical_entity_reference="EtOH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_source(physical_entity_reference="NAD", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="Aldehyde", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="NADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_mediator(physical_entity_reference="ADH", uri_type=eUriType.MODEL_URI) \
        .has_property("OPB:OPB_00237")                       #Chemical flow rate

print(rdf_graph3_1_2_1)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix semsim: <http://bime.uw.edu/semsim/> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/AHD.omex/> .
@prefix local: <http://omex-library.org/AHD.omex/AHDModel.rdf#> .

local:MediatorParticipant0000
    semsim:hasPhysicalEntityReference <http://omex-library.org/AHD.omex/AHDModel.sbml#ADH> .

local:ProcessProperty0000
    bqbiol:isPropertyOf <http://omex-library.org/AHD.omex/AHDModel.sbml#ADHForwardReaction> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:OPB_00237> .

local:SinkParticipant0000
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference <http://omex-library.org/AHD.omex/AHDModel.sbml#Aldehyde> .

local:SinkParticipant0001
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference <http://omex-library.org/AHD.omex/AHDModel.sbml#NADH> .

local:SourceParticipan

In `Example3_1_2_1`, the reaction has two reactants, two products, and one mediating enzyme.
Note that stoichiometry, where appropriate (`sources` and `sinks`), can be specified via the `multiplier` fields.
Like with the physical entity type composite annotations, the `about` method
provides the link between the annotation document we are composing and the SBML source code. Similarly,
the `physical_entity_reference` arguments are metaids to elements within the SBML document. The final optional clause `has_property` indicates that the reaction being simulated has the physical property
of "chemical flow rate".

<left> <h5> 3.1.3 Annotations for a energy differential </h5> </left>

Finally, we provide an example for annotating the electrical potential across the cellular membrane,
specifically resulting from calcium ions. In addition to source and sink, this is annotated with the
OPB term for the Nernst, or reversal potential.

In [7]:
# Example 3_1_3
from pyomexmeta import RDF, eUriType

sbml3_1_3 = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="EnergyDiff" id="EnergyDiff">
    <listOfCompartments>
      <compartment id="cytoplasm" metaid="cytoplasm" spatialDimensions="3" size="1" constant="true"/>
      <compartment id="extracellular" metaid="extracellular" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="Ca_ex" metaid="Ca_ex" compartment="extracellular" initialConcentration="1" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="Ca_cyt" metaid="Ca_cyt" compartment="cytoplasm" initialConcentration="0.3" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="k1" metaid="k1" value="0.1" constant="true"/>
      <parameter id="k2" metaid="k2" value="0.1" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="NernstReversalPotential_in" metaid="NernstReversalPotential_in" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="Ca_ex" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="Ca_cyt" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> cytoplasm </ci>
              <ci> k1 </ci>
              <ci> Ca_ex </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="NernstReversalPotential_out" metaid="NernstReversalPotential_out" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="Ca_cyt" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="Ca_ex" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> extracellular </ci>
              <ci> k2 </ci>
              <ci> Ca_cyt </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>"""
rdf_graph_3_1_3 = RDF()
rdf_graph_3_1_3.set_archive_uri("Example3_1_3.omex")
rdf_graph_3_1_3.set_model_uri("Example3_1_3.sbml")

annot_editor = rdf_graph_3_1_3.to_editor(sbml3_1_3, generate_new_metaids=False, sbml_semantic_extraction=False)

# Ca_cyt: Calcium Ions cytosol
# Ca_ex: Calcium Ions extracellular space
# NernstReversalPotential_in: The metaID of the SBML reaction
# OPB/OPB_01581: Nernst reversal potential
with annot_editor.new_energy_diff() as energy_in:
    energy_in \
     .about("NernstReversalPotential_in", eUriType.MODEL_URI) \
     .add_source(physical_entity_reference="Ca_ex", uri_type=eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Ca_cyt", uri_type=eUriType.MODEL_URI) \
     .has_property("OPB:OPB_01581")

with annot_editor.new_energy_diff() as energy_out:
    energy_out \
     .about("NernstReversalPotential_out", eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Ca_ex", uri_type=eUriType.MODEL_URI) \
     .add_source(physical_entity_reference="Ca_cyt", uri_type=eUriType.MODEL_URI) \
     .has_property("OPB:OPB_01581")

print(rdf_graph_3_1_3)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix semsim: <http://bime.uw.edu/semsim/> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/Example3_1_3.omex/> .
@prefix local: <http://omex-library.org/Example3_1_3.omex/Example3_1_3.rdf#> .

local:EnergyDiffProperty0000
    bqbiol:isPropertyOf <http://omex-library.org/Example3_1_3.omex/Example3_1_3.sbml#NernstReversalPotential_in> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:OPB_01581> .

local:EnergyDiffProperty0001
    bqbiol:isPropertyOf <http://omex-library.org/Example3_1_3.omex/Example3_1_3.sbml#NernstReversalPotential_out> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:OPB_01581> .

local:SinkParticipant0000
    semsim:hasPhysicalEntityReference <http://omex-library.org/Example3_1_3.omex/Example3_1_3.sbml#Ca_cyt> .

local:SinkParticipant0001
    semsim:hasPhysicalEntityReference <http://omex-library.

<left> <h4> 3.2 CellML </h4> </left>


<left> <h5> 3.2.1 Annotations for a Physical Entity </h5> </left>

As mentioned earlier, in CellML one has variables which represent various physical properties but the biological physical entity is implicit. In this example, we demonstrate how to annotate the `Volume` of `blood` in the `Lumen of the left coronary artery` using a snippet of CellML code.

In [8]:
# Example 3_2_1
from pyomexmeta import RDF, eUriType

cellml = """
<model xmlns="http://www.cellml.org/cellml/1.1#" xmlns:cmeta="http://www.cellml.org/metadata/1.0#"
      name="annotation_examples" cmeta:id="annExamples">
  <component name="main">
    <variable cmeta:id="main.Volume" initial_value="100" name="Volume" units="dimensionless" />
    <variable cmeta:id="main.MembraneVoltage" initial_value="-80" name="MembraneVoltage" units="dimensionless" />
    <variable cmeta:id="main.ReactionRate" initial_value="1" name="ReactionRate" units="dimensionless" />
  </component>
</model>
"""

rdf_graph_3_2_1 = RDF()
rdf_graph_3_2_1.set_archive_uri("physical_process.omex")
rdf_graph_3_2_1.set_model_uri("model.cellml")
annot_editor = rdf_graph_3_2_1.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

# OPB:00154 = fluid volume
# FMA:9670 = portion of blood
# FMA:18228 = Lumen of the left coronary artery
with annot_editor.new_physical_entity() as arterial_volume:
    arterial_volume.about("local-entity-0", eUriType.LOCAL_URI) \
                   .identity("FMA:9670") \
                   .is_part_of("FMA:18228") \
                   .has_property("main.Volume", eUriType.MODEL_URI, "opb:OPB_00154")
print(rdf_graph_3_2_1)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/physical_process.omex/> .
@prefix local: <http://omex-library.org/physical_process.omex/model.rdf#> .

<http://omex-library.org/physical_process.omex/model.cellml#main.Volume>
    bqbiol:isPropertyOf local:local-entity-0 ;
    bqbiol:isVersionOf <https://identifiers.org/opb:OPB_00154> .

local:local-entity-0
    bqbiol:is <https://identifiers.org/FMA:9670> ;
    bqbiol:isPartOf <https://identifiers.org/FMA:18228> .





In this example we construct a physical entity annotation for the arterial blood volume. The
argument to the `about` method is the cmetaid for the CellML variable element called `main.Volume`.

       <variable cmeta:id="main.Volume" initial_value="100" name="Volume" units="dimensionless" />

Internally, libOmexMeta creates a new metaid (`local:local-entity-0`) which is `local` to the annotation document we are creating that links the variable with cmetaid `main.Volume` to the information that annotates this physical entity.

    local:local-entity-0
        bqbiol:is <https://identifiers.org/FMA:9670> ;
        bqbiol:isPartOf <https://identifiers.org/FMA:18228> .

Often, the `is_part_of` relation is only important when entities can exist in multiple
compartments and the modeler needs to distinguish between them.

<left> <h5> 3.2.2 Annotations for a Physical Process </h5> </left>

Here we describe the annotation of the CellML version of the conversion of `ethanol` into `aldehyde` as described in 3.1.2. In CellML this reaction is represented by the molar flow rate from the `source` `ethanol` to the `sink` `acetaldehyde` via the `mediator` `alcohol dehydrogenase 1A` within the `cytosol` of a `hepatocyte`.


In [9]:
# Example 3_2_2 
from pyomexmeta import RDF, eUriType

cellml = """
<model xmlns="http://www.cellml.org/cellml/1.1#" xmlns:cmeta="http://www.cellml.org/metadata/1.0#"
      name="annotation_examples" cmeta:id="annExamples">
  <component name="main">
    <variable cmeta:id="main.Volume" initial_value="100" name="Volume" units="dimensionless" />
    <variable cmeta:id="main.MembraneVoltage" initial_value="-80" name="MembraneVoltage" units="dimensionless" />
    <variable cmeta:id="main.ReactionRate" initial_value="1" name="ReactionRate" units="dimensionless" />
  </component>
</model>
"""

rdf_graph_3_2_2 = RDF()
rdf_graph_3_2_2.set_archive_uri("physical_process.omex")
rdf_graph_3_2_2.set_model_uri("model.cellml")
annot_editor = rdf_graph_3_2_2.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

# fma:14515 = hepatocyte
# fma:66836 = portion of cytosol
with annot_editor.new_physical_entity() as cytosol:
    cytosol \
        .about("cytosol", eUriType.LOCAL_URI) \
        .identity("FMA:66836") \
        .is_part_of("FMA:14515")
    
# http://purl.obolibrary.org/obo/PR_000003767 = alcohol dehydrogenase 1A
with annot_editor.new_physical_entity() as mediator:
    mediator \
        .about("mediator", eUriType.LOCAL_URI) \
        .identity("http://purl.obolibrary.org/obo/PR_000003767") \
        .is_part_of("cytosol", eUriType.LOCAL_URI)

# CHEBI:16236 = ethanol
with annot_editor.new_physical_entity() as source:
    source \
        .about("source", eUriType.LOCAL_URI) \
        .identity("CHEBI:16236") \
        .is_part_of("cytosol", eUriType.LOCAL_URI)

# CHEBI:15343 = acetaldehyde
with annot_editor.new_physical_entity() as sink:
    sink \
        .about("sink", eUriType.LOCAL_URI) \
        .identity("CHEBI:15343") \
        .is_part_of("cytosol", eUriType.LOCAL_URI)

# opb:OPB_00592 = chemical molar flow rate
# GO:0004022 = alcohol dehydrogenase (NAD+) activity
with annot_editor.new_physical_process() as reaction_rate:
    reaction_rate \
        .about("process", eUriType.LOCAL_URI) \
        .is_version_of("GO:0004022") \
        .add_source("source", eUriType.LOCAL_URI, multiplier=1) \
        .add_sink("sink", eUriType.LOCAL_URI, multiplier=1) \
        .add_mediator("mediator", eUriType.LOCAL_URI) \
        .has_property(property_about="main.ReactionRate", about_uri_type=eUriType.MODEL_URI, is_version_of="opb:OPB_00592")

print(rdf_graph_3_2_2)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix semsim: <http://bime.uw.edu/semsim/> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/physical_process.omex/> .
@prefix local: <http://omex-library.org/physical_process.omex/model.rdf#> .

<http://omex-library.org/physical_process.omex/model.cellml#main.ReactionRate>
    bqbiol:isPropertyOf local:process ;
    bqbiol:isVersionOf <https://identifiers.org/opb:OPB_00592> .

local:MediatorParticipant0000
    semsim:hasPhysicalEntityReference local:mediator .

local:SinkParticipant0000
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference local:sink .

local:SourceParticipant0000
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference local:source .

local:cytosol
    bqbiol:is <https://identifiers.org/FMA:66836> ;
    bqbiol:isPartOf <https://identifiers.org/FMA:14515> .


<left> <h5> 3.2.3 Annotations for a energy differential </h5> </left>

We show here an example of annotating an Energy differential, specifically the electrical potential across a `membrane` of a pancreatic islet.

In [10]:
# Example 3_2_3
from pyomexmeta import RDF, eUriType

cellml = """
<model xmlns="http://www.cellml.org/cellml/1.1#" xmlns:cmeta="http://www.cellml.org/metadata/1.0#"
      name="annotation_examples" cmeta:id="annExamples">
  <component name="main">
    <variable cmeta:id="main.Volume" initial_value="100" name="Volume" units="dimensionless" />
    <variable cmeta:id="main.MembraneVoltage" initial_value="-80" name="MembraneVoltage" units="dimensionless" />
    <variable cmeta:id="main.ReactionRate" initial_value="1" name="ReactionRate" units="dimensionless" />
  </component>
</model>
"""

rdf_graph_3_2_3 = RDF()
rdf_graph_3_2_3.set_archive_uri("physical_process.omex")
rdf_graph_3_2_3.set_model_uri("model.cellml")
annot_editor = rdf_graph_3_2_3.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

# FMA:16016 = Pancreatic islet
# FMA:9672 = Intercellular matrix
with annot_editor.new_physical_entity() as matrix:
    matrix \
        .about("intercellular-matrix", eUriType.LOCAL_URI) \
        .identity("FMA:9672") \
        .is_part_of("FMA:16016")
    
# CHEBI:24870 - ion
with annot_editor.new_physical_entity() as sink:
    sink \
        .about("sink", eUriType.LOCAL_URI) \
        .identity("CHEBI:24870") \
        .is_part_of("intercellular-matrix", eUriType.LOCAL_URI)

# FMA:70586 = Type B cell of pancreatic islet
# FMA:66836 = Portion of cytosol
with annot_editor.new_physical_entity() as cytosol:
    cytosol \
        .about("cytosol", eUriType.LOCAL_URI) \
        .identity("FMA:66836") \
        .is_part_of("FMA:70586")
    
# CHEBI:24870 - ion
with annot_editor.new_physical_entity() as source:
    source \
        .about("source", eUriType.LOCAL_URI) \
        .identity("CHEBI:24870") \
        .is_part_of("cytosol", eUriType.LOCAL_URI)

# opb:OPB_00506 = Electrical potential
with annot_editor.new_energy_diff() as energy_diff:
    energy_diff \
        .about("energy-diff", eUriType.LOCAL_URI) \
        .add_source("source", eUriType.LOCAL_URI) \
        .add_sink("sink", eUriType.LOCAL_URI) \
        .has_property("main.MembraneVoltage", eUriType.MODEL_URI, "opb:OPB_00506")
    
print(rdf_graph_3_2_3)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix semsim: <http://bime.uw.edu/semsim/> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/physical_process.omex/> .
@prefix local: <http://omex-library.org/physical_process.omex/model.rdf#> .

<http://omex-library.org/physical_process.omex/model.cellml#main.MembraneVoltage>
    bqbiol:isPropertyOf local:energy-diff ;
    bqbiol:isVersionOf <https://identifiers.org/opb:OPB_00506> .

local:SinkParticipant0000
    semsim:hasPhysicalEntityReference local:sink .

local:SourceParticipant0000
    semsim:hasPhysicalEntityReference local:source .

local:cytosol
    bqbiol:is <https://identifiers.org/FMA:66836> ;
    bqbiol:isPartOf <https://identifiers.org/FMA:70586> .

local:energy-diff
    semsim:hasSinkParticipant local:SinkParticipant0000 ;
    semsim:hasSourceParticipant local:SourceParticipant0000 .

local:intercellular-matri

<center> <h2> Part B: libOmexMeta Examples </h2> </center>
While the preceding section is designed to be more tutorial based to
guide users around the libOmexMeta library, this section is designed to be
example led, with an emphasis on how one is likely to use libOmexMeta in conjuction with `libcombine`, for creating COMBINE archives.

In these examples, we use a simple SBML model that simulates the traditional Michaelis-Menten reaction schema.

In [11]:
import sys
import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType

# globally used for the remainder of this document.
SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath('__file__'))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("18-09-2020")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source("S", eUriType.MODEL_URI,1.0) \
                .add_source("E", eUriType.MODEL_URI,1.0) \
                .add_sink("ES", eUriType.MODEL_URI,1.0)

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink("S", eUriType.MODEL_URI,1.0) \
                .add_sink("E", eUriType.MODEL_URI,1.0) \
                .add_source("ES", eUriType.MODEL_URI,1.0)

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink("E", eUriType.MODEL_URI,1.0) \
                .add_sink("P", eUriType.MODEL_URI,1.0) \
                .add_source("ES", eUriType.MODEL_URI,1.0)

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source("S", eUriType.MODEL_URI) \
                .add_source("E", eUriType.MODEL_URI) \
                .add_sink("ES", eUriType.MODEL_URI)

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # Convenience flag. When True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath('__file__')), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]

Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/ModelLevelAnnotations.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/ModelLevelAnnotations.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PersonalInformation.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PersonalInformation.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/SingularAnnotations.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/SingularAnnotations.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PhysicalEntity.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PhysicalEntity.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PhysicalProcess.omex"
Archive written to "/Users/ciaranwelsh/Downloads/libomexmeta-SI-master/PhysicalProcess.omex"
Archive written to "/Users/ciaranwelsh/Downl